In [1]:
import requests
from bs4 import BeautifulSoup
import pickle

In [29]:
urlbase = pickle.load(open('urlbase.pickle', 'rb'))

hrefs = []
for year in range(2010, 2021):
    count_hrefs = 0
    for month in range(1, 13):
        # assemble URL of current month to extract hrefs of events
        url = urlbase+"/concerts?field_date_value%5Bvalue%5D%5Bmonth%5D="+str(month)
        url += "&field_date_value%5Bvalue%5D%5Byear%5D="+str(year)
        url += "&tid=All"
        
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        for event in soup.find_all('div', class_="month mini-day-on"):
            hrefs.append(event.find_all('a')[0].get('href'))
            count_hrefs += 1
            
    print(year, "found", count_hrefs)
            
print(hrefs)

2010 found 46
2011 found 38
2012 found 41
2013 found 52
2014 found 36
2015 found 44
2016 found 49
2017 found 31
2018 found 34
2019 found 25
2020 found 2
['/events/holst-singers-hyperion-cd-recording', '/events/polyphony-scotland', '/events/poland', '/events/holst-singers-twickenham', '/events/holst-singers-cambridge', '/events/danish-radio-vocal-ensemble-sweden', '/events/danish-radio-vocal-ensemble-danish-radio-4', '/events/danish-radio-vocal-ensemble-danish-radio-5', '/events/choir-trinity-college-cambridge-cambridge-6', '/events/trinity-college-choir-cambridge-nocturne-0', '/events/trinity-college-choir-cambridge-nocturne-1', '/events/odense-symphony-orchestra-denmark', '/events/polyphony-bach-st-john-passion-6', '/events/polyphony-cambridge', '/events/polyphony-passion-and-resurrection-0', '/events/polyphony-passion-and-resurrection-1', '/events/city-london-sinfonia-choir-trinity-college-cambridge-aldeburgh', '/events/holst-singers-oxford-0', '/events/holst-singers-bach-motets-and-

In [125]:
import pickle
pickle.dump(hrefs, open('hrefs.pickle', 'wb'))

In [3]:
import pickle
import sys
hrefs = pickle.load(open('hrefs.pickle', 'rb'))

In [4]:
from tqdm import tqdm
urlbase = pickle.load(open('urlbase.pickle', 'rb'))

lines, line_count = [], 0
for href in tqdm(hrefs):
    url = urlbase + href

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    suptitle = soup.find_all('h1', class_="title gutter")[0].contents[0]

    fields = soup.find_all('div', class_="field-items")

    try:
        location, title = '', ''
        date = soup.find_all('span', class_="date-display-single")[0].get('content')
        for i in range(len(fields)):
            paragraphs = fields[i].find_all('p')
            if len(paragraphs) > 0: 
                location = paragraphs[-1].contents[0]
                if len(paragraphs) > 1: title = paragraphs[0].renderContents() # TODO: strip all HTML tags in these
                break
                
    except IndexError as e:
        print(url)
        print(soup.find_all('p')[0])
        sys.exit()

    artists, composers = [], []
    composerMode = False # whether we're in the artist (False) or composer (True) column

    subfields = [field.find_all('div', class_="field-item even")[0] if len(field.find_all('div', class_="field-item even")) > 0 else None for field in fields[2:]]
    for subfield in subfields:
        if subfield is not None: 
            subfield = subfield.next # enter subfield
            if str(subfield).find('composer') > -1: composerMode = True
            if str(subfield).find('<') == -1 and len(subfield) > 1: 
                if composerMode: composers.append(subfield)
                else:            artists.append(subfield) # TODO: append things like Director/soprano/alto/tenor/bass to the previous element

    lines.append([date, suptitle, title, location, ','.join(artists), ','.join(composers)])
    line_count += 1
    
print(line_count, "lines added successfully")

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it]

1 lines added successfully


In [ ]:
# save to file
import codecs # write as UTF8 - TODO: check that this does what it says

with codecs.open('trinity.csv', 'a', "utf-8") as the_file:
    the_file.write('date;title;location;artists;composers\n')
    for line in lines:
        the_file.write(';'.join(line)+'\n')
        
print("File successfully saved")